# Classifing last names

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import random

## Dataset
`wget https://github.com/hunkim/PyTorchZeroToAll/blob/master/data/names_train.csv.gz`

`wget https://github.com/hunkim/PyTorchZeroToAll/blob/master/data/names_test.csv.gz`

In [4]:
PATH = Path("/data2/yinterian/name_dataset/")
list(PATH.iterdir())

[PosixPath('/data2/yinterian/name_dataset/names_test.csv'),
 PosixPath('/data2/yinterian/name_dataset/names_train.csv.gz'),
 PosixPath('/data2/yinterian/name_dataset/names_train.csv'),
 PosixPath('/data2/yinterian/name_dataset/names_test.csv.gz')]

In [5]:
! head /data2/yinterian/name_dataset/names_train.csv

### One hot encoding

In [88]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv(PATH/"names_train.csv", header=None)

In [84]:
letters = [list(l) for l in df[0].values]
vocab = sorted(list(set(np.concatenate(np.array(letters)))))
vocab[:10]

[' ', "'", ',', 'A', 'B', 'C', 'D', 'E', 'F', 'G']

In [85]:
vocab2id = {key:i for i, key in enumerate(vocab)}
#vocab2id

In [91]:
class NameDataset(Dataset):
    def __init__(self, path, vocab2id):
        self.df = pd.read_csv(path, header=None)
        self.labels = sorted(df[1].unique())
        self.label2id = {key:i for i, key in enumerate(labels)}
        self.x = df[0].values
        self.y = [self.label2id[l] for l in df[1].values]
        self.vocab2id = vocab2id
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = list(self.x[idx])
        x = [self.vocab2id[k] for k in x]
        return x, self.y[idx]

In [50]:
train = NameDataset(PATH/"names_train.csv")

In [52]:
x,y = train[1]
print(x,y)

['A', 'j', 'd', 'r', 'n', 'a', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'] 2


## Model

In [54]:
class Seq2SeqSRNN(nn.Module):

    def __init__(self, n_features=2, n_hiddlen=10, classes=18):
        super().__init__()
        self.l_in = nn.Linear(n_features, n_hidden) # multiple features in
        self.l_hidden = nn.Linear(2*n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, classes) # out is just temperature
        
    def forward(self, x):
        bash_size = x.shape[0]
        h = Variable(torch.zeros(bash_size, n_hidden).cuda())
        for i in range(x.shape[1]):
            x_i = x[:,i]
            inp = F.relu(self.l_in(x_i))
            h_inp = torch.cat([h, inp],1)
            h = F.tanh(self.l_hiddlen(h_inp))
        return self.l_out(h)